In [76]:
anomaly_proportion=0.6
objeto="tile"

In [77]:
import random
random.seed(42)

## Distribución de los datos con centroide

In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

class Centroid:
    def __init__(self, k):
        self.k = k
        self.centroid = None
        self.threshold = None

    def fit(self, embeddings):
        # Calcular el centroide
        self.centroid = torch.mean(embeddings, dim=0)

        # Calcular las distancias al centroide
        distances = torch.norm(embeddings - self.centroid, dim=1)

        # Calcular la media y desviación estándar de las distancias
        mean_dist = torch.mean(distances)
        std_dist = torch.std(distances)

        # Definir el umbral
        self.threshold = mean_dist + self.k * std_dist

    def predict(self, embeddings):
        if self.centroid is None or self.threshold is None:
            raise ValueError("Debe llamar al método fit antes de predecir.")

        # Calcular las distancias al centroide
        distances = torch.norm(embeddings - self.centroid, dim=1)

        # Detectar anomalías
        predicted_labels = (distances > self.threshold).int()
        return predicted_labels,distances,self.threshold



In [79]:
from sklearn.metrics import accuracy_score, f1_score,roc_auc_score
from sklearn.model_selection import train_test_split
import random
import os
names=["tus_embeddings.csv","embeddings.csv","dataset_embeddings_encoder.csv","dataset_embeddings_encoder_resnet.csv"]

for name in names:
    # Cargar el dataset desde el archivo CSV
    # ==========================
    # Cargar los Embeddings
    # ==========================
    df = pd.read_csv(name)

    # Mostrar las primeras filas del dataset
    print(df.head(52))
    random.seed(42)  # Para reproducibilidad

    # Dividir el dataset en conjuntos de entrenamiento y prueba
    X = df[df.columns[:-1]]  # Todas las columnas excepto la última
    y = df["label"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    k=[0.02,0.05,0.1,0.2,0.5,1,5]

    # Mostrar las primeras filas del conjunto de entrenamiento
    print("Conjunto de entrenamiento:")
    print(X_train.head(10))
    print("Conjunto de prueba:")
    print(X_test.head(10))

    print(f"Total de imagenes en el conjunto de entrenamiento: {len(X_train)}")
    print(f"Total de imagenes en el conjunto de prueba: {len(X_test)}")
    folder_name=f"experimento_real/{objeto}/distance/{name}"

    os.makedirs(folder_name, exist_ok=True)

    resultados=[]


    print("Anomaly Detector")
    for i in k:

        model=Centroid(i)

        X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
        model.fit(X_train_tensor)

        X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)


        preds,distances,treshold = model.predict(X_test_tensor)

    



        acc= accuracy_score(y_test, preds)
        f1 = f1_score(y_test, preds)
        auc= roc_auc_score(y_test,distances)
        print("Accuracy:", acc)
        print("F1 Score:", f1)
        print("Area bajo la curva ROC",auc)

        resultados.append({
                'k': i,
                'accuracy': acc,
                'f1_score': f1,
                'roc_auc': auc
            })
        
        # Crear lista de pares (distancia, etiqueta, predicciones)
        dist_label_pairs = list(zip(distances, y_test, preds))

        # Ordenar por distancia descendente
        dist_label_pairs_sorted = sorted(dist_label_pairs, key=lambda x: x[0], reverse=True)

        print("Distancias ordenadas (de mayor a menor) con sus etiquetas:")
        for dist, label, pred  in dist_label_pairs_sorted:
            print(f"Distancia: {dist:.4f}, Etiqueta: {label}, Predicha:{pred}")

        print(f"El treshold es {treshold}")
    # Guardar resultados en Excel
    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_excel(os.path.join(folder_name, 'metricas_por_k.xlsx'), index=False)

    # Guardar embeddings
    pd.DataFrame(X_train_tensor.numpy()).to_csv(os.path.join(folder_name, 'X_train.csv'), index=False)
    pd.DataFrame(X_test_tensor.numpy()).to_csv(os.path.join(folder_name, 'X_test.csv'), index=False)

    # Guardar etiquetas verdaderas
    pd.DataFrame({'y_test': y_test}).to_csv(os.path.join(folder_name, 'y_test.csv'), index=False)

    print(f"Resultados guardados en la carpeta: {folder_name}")

      0    1    2    3    4    5    6    7    8    9  ...  3063  3064  3065  \
0   154  166  150  127  134  121  136  144  131  144  ...   119   124   118   
1   129  131  118  114  116  106  126  129  118  107  ...   118   120   114   
2   144  146  142  128  130  126  145  149  146  135  ...   141   147   138   
3   117  118  114  138  141  136  115  116  113  109  ...   127   131   124   
4   120  123  112  129  132  121  138  140  129  140  ...   129   129   125   
5   101  103   96   95   95   90  124  127  119  122  ...    95    94    91   
6   123  122  117  132  131  126  140  140  135  139  ...   113   114   105   
7   107  112  103  108  113  104  115  121  112  115  ...   107   110   105   
8   132  139  129  127  133  123  132  137  127  147  ...   103   106   102   
9   133  138  124  124  128  116   85   86   79  111  ...   136   138   134   
10  141  150  138  142  151  140  139  147  137  159  ...   109   113   108   
11  118  117  112  133  134  129  120  120  114   92

## Método Antonio

In [80]:
import torch

class GDAOneClassTorch:
    def fit(self, X, threshold_param=1.0):
        """
        Ajusta el modelo GDA para una sola clase usando PyTorch.
        :param X: Tensor de forma (n_samples, n_features)
        """
        self.mu = X.mean(dim=0)
        self.centered = X - self.mu
        self.sigma = torch.matmul(self.centered.T, self.centered) / X.shape[0]     #calcula la matriz de covarianzas

        # Regularización para evitar matriz singular
        epsilon = 1e-3
        self.sigma += epsilon * torch.eye(self.sigma.shape[0])
        self.inv_sigma = torch.inverse(self.sigma)
        self.det_sigma = torch.det(self.sigma)

        scores= self.score(X)


        mean_score = scores.mean()
        std_score = scores.std()
        self.threshold = mean_score - threshold_param * std_score

    def score(self, X):
        scores=[]
        for i in range(len(X)):
            centered = X[i] - self.mu
            tmp = torch.matmul(centered, self.inv_sigma)
            quad_form = (tmp * centered).sum()
            scores.append(-0.5 * quad_form)
        return torch.tensor(scores)

    def predict(self, X):
        scores = self.score(X)
        preds=[] 
        for i in range(len(scores)):
            pred= 1 if scores[i]< self.threshold else 0
            preds.append((i, pred))

        return preds,self.threshold



    def print_parameters(self):
        """
        Imprime el vector de medias y la matriz de covarianza.
        """
        print("Vector de medias (mu):")
        print(self.mu)
        print("\nMatriz de covarianza (sigma):")
        print(self.sigma)

    def print_score(self, X):
      """
      Imprime y retorna la densidad de cada punto en X.
      :param X: Tensor (n_samples, n_features)
      :return: Diccionario {índice: densidad}
      """
      probs = self.score(X)
      scores = dict()

      for i, p in enumerate(probs):
          valor = p.item()
          print(f"Densidad del punto {i}: {valor:.6f}")
          scores[i] = valor

      return scores


In [81]:
from sklearn.metrics import accuracy_score,f1_score
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import random

names=["tus_embeddings.csv","embeddings.csv","dataset_embeddings_encoder.csv","dataset_embeddings_encoder_resnet.csv"]

for name in names:

    df = pd.read_csv(name)

    random.seed(42)  # Para reproducibilidad

    # Dividir el dataset en conjuntos de entrenamiento y prueba
    X = df[df.columns[:-1]]  # Todas las columnas excepto la última
    y = df["label"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # Mostrar las primeras filas del conjunto de entrenamiento
    print("Conjunto de entrenamiento:")
    print(X_train.head(10))
    print("Conjunto de prueba:")
    print(X_test.head(10))

    print(f"Total de imagenes en el conjunto de entrenamiento: {len(X_train)}")
    print(f"Total de imagenes en el conjunto de prueba: {len(X_test)}")

    folder_name=f"experimento_real/{objeto}/GDA/{name}"

    os.makedirs(folder_name, exist_ok=True)

    resultados=[]

    
    k=[0.1,0.5,1,2,5,10,20,30,40,50,60,70,80,100]

    for i in k:

        model = GDAOneClassTorch()
        X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
        model.fit(X_train_tensor,i)




        X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)


        preds_with_index,treshold= model.predict(X_test_tensor)
        '''
        print(model.threshold)
        preds = torch.tensor(preds)
        print(preds.shape)
        
        # Paso 3: imprimir por pantalla (predicción y etiqueta real)
        for i, (p, real) in enumerate(zip(preds[1], y_test)):
            print(f"Ejemplo {i}: Predicción = {p.item()}, Etiqueta real = {real}")


        '''
        # Extraer predicciones en el mismo orden que y_test
        preds = [pred for x, pred in preds_with_index]
        scores=(model.score(X_test_tensor))**2

        f1 = f1_score(y_test.to_numpy(), preds)
        acc = accuracy_score(y_test.to_numpy(), preds)
        auc= roc_auc_score(y_test,scores)
        print("F1 Score:", f1)
        print("Accuracy:", acc)
        print("Area bajo la curva ROC",auc)

        resultados.append({
                'k': i,
                'accuracy': acc,
                'f1_score': f1,
                'roc_auc': auc
            })
        # Crear lista de pares (distancia, etiqueta,predicciones)
        score_label_pairs = list(zip(model.score(X_test_tensor), y_test,preds))

        # Ordenar por distancia descendente
        score_label_pairs_sorted = sorted(score_label_pairs, key=lambda x: x[0], reverse=True)

        print("Distancias ordenadas (de mayor a menor) con sus etiquetas:")
        for score, label, pred in score_label_pairs_sorted:
            print(f"Distancia: {score:.4f}, Etiqueta: {label}, Predicha:{pred}")
        print(f"El treshold es {treshold}")
    # Guardar resultados en Excel
    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_excel(os.path.join(folder_name, 'metricas_por_k.xlsx'), index=False)

    # Guardar embeddings
    pd.DataFrame(X_train_tensor.numpy()).to_csv(os.path.join(folder_name, 'X_train.csv'), index=False)
    pd.DataFrame(X_test_tensor.numpy()).to_csv(os.path.join(folder_name, 'X_test.csv'), index=False)

    # Guardar etiquetas verdaderas
    pd.DataFrame({'y_test': y_test}).to_csv(os.path.join(folder_name, 'y_test.csv'), index=False)

    print(f"Resultados guardados en la carpeta: {folder_name}")



        



Conjunto de entrenamiento:
       0    1    2    3    4    5    6    7    8    9  ...  3062  3063  3064  \
176  147  147  139  109  109  102  132  132  124  126  ...    98   113   116   
78   158  165  153  157  164  154  150  156  146  140  ...   113   133   136   
90   134  138  126   91   93   87   91   92   88  134  ...   134   142   143   
16    95   96   88  120  124  115  114  117  109  120  ...   142   113   117   
66    99  102   94  100  102   94   98  100   93  135  ...    85    88    88   
126  137  146  132  129  138  127  105  111  102  131  ...   121   138   145   
7    107  112  103  108  113  104  115  121  112  115  ...   139   107   110   
219  101  105   96  105  110  102   99  103   96  138  ...   124   136   142   
344  143  148  136  155  162  151  128  132  124  102  ...    96   105   105   
331  161  161  155  142  142  138  132  131  129  131  ...   111   113   114   

     3065  3066  3067  3068  3069  3070  3071  
176   106   136   142   129   115   118   10

## Nuevo Metodo Antonio

In [82]:
class ZScoreThreshold:
    def fit(self, X):
        """
        Calcula media y desviación estándar por columna.
        X: tensor (n_samples, n_features)
        """
        self.mu = X.mean(dim=0)
        self.std = X.std(dim=0)
        self.std[self.std == 0] = 1e-6  # evita divisiones por cero

    def score(self, X):
        """
        Retorna el número de columnas fuera de lo normal por muestra.
        """
        z_scores = (X - self.mu) / self.std
        return (z_scores.abs() > 1.96).sum(dim=1)  # puedes parametrizar el umbral

    def predict(self, X, percentage=0.1):
        """
        Marca como anomalía si más de un porcentaje de columnas están fuera del estándar.
        
        percentage: porcentaje de columnas que deben estar fuera del estándar para ser considerada anomalía.
        """
        # Calcula el umbral de columnas basado en el porcentaje
        num_columns = int(percentage * X.shape[1])  # Número de columnas según el porcentaje
        
        scores = self.score(X)
        return (scores > num_columns).int(),scores  # Marca como anomalía si supera el umbral de columnas
    


In [83]:
from sklearn.metrics import accuracy_score,f1_score,roc_curve

names=["tus_embeddings.csv","embeddings.csv","dataset_embeddings_encoder.csv","dataset_embeddings_encoder_resnet.csv"]

for name in names:

    df = pd.read_csv(name)

    random.seed(42)  # Para reproducibilidad

    # Dividir el dataset en conjuntos de entrenamiento y prueba
    X = df[df.columns[:-1]]  # Todas las columnas excepto la última
    y = df["label"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # Mostrar las primeras filas del conjunto de entrenamiento
    print("Conjunto de entrenamiento:")
    print(X_train.head(10))
    print("Conjunto de prueba:")
    print(X_test.head(10))

    print(f"Total de imagenes en el conjunto de entrenamiento: {len(X_train)}")
    print(f"Total de imagenes en el conjunto de prueba: {len(X_test)}")

    folder_name=f"experimento_real/{objeto}/columnas/{name}"

    os.makedirs(folder_name, exist_ok=True)

    resultados=[]

    k=[0.01,0.05,0.1,0.2,0.25,0.3]

    for i in k:

        model=ZScoreThreshold()
        X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
        model.fit(X_train_tensor)

        X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
        preds,scores = model.predict(X_test_tensor,i)
        print(preds.shape)
        '''
        # Paso 3: imprimir por pantalla (predicción y etiqueta real)
        for i, (p, real) in enumerate(zip(preds, y_test)):
            print(f"Ejemplo {i}: Predicción = {p.item()}, Etiqueta real = {real}")
        '''

        acc= accuracy_score(y_test, preds)
        f1 = f1_score(y_test, preds)
        auc= roc_auc_score(y_test,scores)
        print("Accuracy:", acc)
        print("F1 Score:", f1)
        print("Area bajo la curva ROC")

        resultados.append({
                'k': i,
                'accuracy': acc,
                'f1_score': f1,
                'roc_auc': auc
            })
        
        # Crear lista de pares (distancia, etiqueta)
        score_label_pairs = list(zip(scores, y_test))

        # Ordenar por distancia descendente
        score_label_pairs_sorted = sorted(score_label_pairs, key=lambda x: x[0], reverse=True)

        print("Distancias ordenadas (de mayor a menor) con sus etiquetas:")
        for score, label in score_label_pairs_sorted:
            print(f"Columnas: {score:.4f}, Etiqueta: {label}")
        
    # Guardar resultados en Excel
    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_excel(os.path.join(folder_name, 'metricas_por_k.xlsx'), index=False)

    # Guardar embeddings
    pd.DataFrame(X_train_tensor.numpy()).to_csv(os.path.join(folder_name, 'X_train.csv'), index=False)
    pd.DataFrame(X_test_tensor.numpy()).to_csv(os.path.join(folder_name, 'X_test.csv'), index=False)

    # Guardar etiquetas verdaderas
    pd.DataFrame({'y_test': y_test}).to_csv(os.path.join(folder_name, 'y_test.csv'), index=False)

    print(f"Resultados guardados en la carpeta: {folder_name}")



Conjunto de entrenamiento:
       0    1    2    3    4    5    6    7    8    9  ...  3062  3063  3064  \
176  147  147  139  109  109  102  132  132  124  126  ...    98   113   116   
78   158  165  153  157  164  154  150  156  146  140  ...   113   133   136   
90   134  138  126   91   93   87   91   92   88  134  ...   134   142   143   
16    95   96   88  120  124  115  114  117  109  120  ...   142   113   117   
66    99  102   94  100  102   94   98  100   93  135  ...    85    88    88   
126  137  146  132  129  138  127  105  111  102  131  ...   121   138   145   
7    107  112  103  108  113  104  115  121  112  115  ...   139   107   110   
219  101  105   96  105  110  102   99  103   96  138  ...   124   136   142   
344  143  148  136  155  162  151  128  132  124  102  ...    96   105   105   
331  161  161  155  142  142  138  132  131  129  131  ...   111   113   114   

     3065  3066  3067  3068  3069  3070  3071  
176   106   136   142   129   115   118   10

## Reconstruccion

In [84]:
import torch
import torch.nn as nn
import torchvision.models as models


class ViewLayer(nn.Module):
    def __init__(self, shape):
        super().__init__()
        self.shape = shape

    def forward(self, x):
        return x.view(x.size(0), *self.shape)

class ResNetAutoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        resnet = models.resnet50(pretrained=True)

        # Compacto: todas las capas convolucionales hasta layer4
        self.encoder = nn.Sequential(*list(resnet.children())[:8],
                                     nn.MaxPool2d(kernel_size=4,stride=1),
                                     nn.Flatten(),
                                     nn.Linear(2048, 1024),
                                        nn.ReLU(),
                                        nn.Dropout(0.2),
                                     nn.Linear(1024, 256),
                                        nn.ReLU(),
                                        nn.Dropout(0.2),
                                        nn.Linear(256, 128),
                                        nn.ReLU())



        # Decoder: invertir el proceso usando conv transpuestas
        self.decoder = nn.Sequential(
            # Primero, expandimos [B, 64] → [B, 256]
            nn.Linear(128, 256),
            nn.ReLU(),

            # Expandimos más: [B, 256] → [B, 2048]
            nn.Linear(256, 2048*4*4),
            nn.ReLU(),
            ViewLayer((2048, 4, 4)),  # Cambiar la forma a [B, 2048, 4, 4]
            # Ahora, aplicamos las capas


            nn.ConvTranspose2d(2048, 512, kernel_size=4, stride=2, padding=1),  # -> [B, 512, 8, 8]
            nn.ReLU(),
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),    # -> [B, 256, 32, 32]
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),     # -> [B, 64, 64, 64]
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),      # -> [B, 64, 128, 128]
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),       # -> [B, 3, 128, 128]
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Prueba con un tensor de entrada (batch_size=1, canal=3, 28x28)
x_test = torch.randn(1, 3, 128, 128)

# Inicializar modelo y obtener salida
model = ResNetAutoencoder()
output = model(x_test)
print(model.state_dict)
output_encoder=model.encoder(x_test)
# Imprimir dimensiones de salida
print("Dimensión final de salida:", output.shape)  # Esperado: (1, 1, 28, 28)
print("Dimensión final de salida encoder:", output_encoder.shape)  # Esperado: (1, 1, 28, 28)



/opt/python/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/python/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<bound method Module.state_dict of ResNetAutoencoder(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsa

In [85]:
import torch.optim as optim

class AutoencoderAnomalyDetector:
    def __init__(self, autoencoder: nn.Module, device='cpu', z_score=1.96):
        self.autoencoder = autoencoder.to(device)
        self.threshold = None
        self.device = device
        self.z_score = z_score
        self.train_losses = []
        self.val_losses = []

    def _train_epoch(self, train_loader, lr=1e-4):
        self.autoencoder.train()
        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.autoencoder.parameters(), lr=lr)
        total_loss = 0.0

        for x in train_loader:
            x = x.to(self.device)
            x_hat = self.autoencoder(x)
            loss = criterion(x_hat, x)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        epoch_loss = total_loss / len(train_loader)
        self.train_losses.append(epoch_loss)
        return epoch_loss

    def _validate_epoch(self, val_loader):
        self.autoencoder.eval()
        criterion = nn.MSELoss()
        total_loss = 0.0

        with torch.no_grad():
            for x in val_loader:
                x = x.to(self.device)
                x_hat = self.autoencoder(x)
                loss = criterion(x_hat, x)
                total_loss += loss.item()

        epoch_loss = total_loss / len(val_loader)
        self.val_losses.append(epoch_loss)
        return epoch_loss

    def compute_threshold(self, loader):
        self.autoencoder.eval()
        all_errors = []

        with torch.no_grad():
            for x in loader:
                x = x.to(self.device)
                x_hat = self.autoencoder(x)
                loss = nn.MSELoss(reduction='none')(x_hat, x)
                per_sample_error = loss.view(loss.size(0), -1).mean(dim=1)
                all_errors.extend(per_sample_error.cpu().numpy())

        all_errors = np.array(all_errors)
        mean_error = all_errors.mean()
        std_error = all_errors.std()
        self.threshold = mean_error + self.z_score * std_error

        print(f"Umbral de reconstrucción (media + {self.z_score}*std): {self.threshold:.6f}")
        return self.threshold

    def eval(self, test_loader):
        self.autoencoder.eval()
        preds = []
        errors = []

        with torch.no_grad():
            for x in test_loader:
                x = x.to(self.device)
                x_hat = self.autoencoder(x)
                loss = nn.MSELoss(reduction='none')(x_hat, x)
                per_sample_error = loss.view(loss.size(0), -1).mean(dim=1)
                errors.extend(per_sample_error.cpu().numpy())
                preds.extend((per_sample_error > self.threshold).int().cpu().numpy())

        return preds, errors


In [86]:
import random
import pandas as pd
from sklearn.model_selection import train_test_split


# Cargar el dataset desde el archivo CSV
dataset_path = f"imagenes_reales/dataset_labels.csv"
df = pd.read_csv(dataset_path)

# Mostrar las primeras filas del dataset
print(df.head(52))
random.seed(42)  # Para reproducibilidad

# Dividir el dataset en conjuntos de entrenamiento y prueba
X = df["filename"]
y = df["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Mostrar las primeras filas del conjunto de entrenamiento
print("Conjunto de entrenamiento:")
print(X_train.head(10))
print("Conjunto de prueba:")
print(X_test.head(10))

print(f"Total de imagenes en el conjunto de entrenamiento: {len(X_train)}")
print(f"Total de imagenes en el conjunto de prueba: {len(X_test)}")

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

               filename  label
0   train_img_00000.png      0
1   train_img_00001.png      0
2   train_img_00002.png      0
3   train_img_00003.png      0
4   train_img_00004.png      0
5   train_img_00005.png      0
6   train_img_00006.png      0
7   train_img_00007.png      0
8   train_img_00008.png      0
9   train_img_00009.png      0
10  train_img_00010.png      0
11  train_img_00011.png      0
12  train_img_00012.png      0
13  train_img_00013.png      0
14  train_img_00014.png      0
15  train_img_00015.png      0
16  train_img_00016.png      0
17  train_img_00017.png      0
18  train_img_00018.png      0
19  train_img_00019.png      0
20  train_img_00020.png      0
21  train_img_00021.png      0
22  train_img_00022.png      0
23  train_img_00023.png      0
24  train_img_00024.png      0
25  train_img_00025.png      0
26  train_img_00026.png      0
27  train_img_00027.png      0
28  train_img_00028.png      0
29  train_img_00029.png      0
30  train_img_00030.png      0
31  trai

In [87]:
import torchvision.transforms as transforms
import torch
from  PIL import Image
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.Lambda(lambda img: img.convert("RGB")),  # <--- convierte a RGB
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])



# Cargar y transformar todas las imágenes normales
image_tensors = []
for path in X_train.tolist():
    img = Image.open(f"imagenes_reales/{str(path)}")
    img = transform(img)
    image_tensors.append(img)

X_train_resnet = torch.stack(image_tensors)

# Cargar y transformar todas las imágenes normales
image_tensors = []
for path in X_val.tolist():
    img = Image.open(f"imagenes_reales/{str(path)}")
    img = transform(img)
    image_tensors.append(img)

X_val_resnet = torch.stack(image_tensors)


# Cargar y transformar todas las imágenes normales
image_tensors = []
for path in X_test:
    img = Image.open(f"imagenes_reales/{str(path)}")
    img = transform(img)
    image_tensors.append(img)

X_test_resnet = torch.stack(image_tensors)

In [88]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import numpy as np
import os

# Selección de dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hiperparámetros
batch_size = 128
learning_rate = 0.0001
num_epochs = 100
patience = 10



# DataLoaders
train_loader = torch.utils.data.DataLoader(X_train_resnet, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(X_val_resnet, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(X_test_resnet, batch_size=batch_size, shuffle=False)

# Crear y entrenar detector
autoencoder = ResNetAutoencoder()
detector = AutoencoderAnomalyDetector(autoencoder, device=device)

print("Entrenando Autoencoder con validación y early stopping...")

best_val_loss = float('inf')
epochs_without_improvement = 0
train_losses, val_losses = [], []

# Scheduler: reduce a la mitad cada 5 épocas
scheduler = StepLR(optim.Adam(autoencoder.parameters(),lr=learning_rate), step_size=5, gamma=0.5)

for epoch in range(num_epochs):
    train_loss = detector._train_epoch(train_loader, lr=learning_rate)
    val_loss = detector._validate_epoch(val_loader)

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    scheduler.step()

    print(f"Época [{epoch+1}/{num_epochs}] - Pérdida entrenamiento: {train_loss:.6f} | Pérdida validación: {val_loss:.6f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_without_improvement = 0
        torch.save(detector.autoencoder.state_dict(), "autoencoder_resnet_best.pth")
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print(f"Deteniendo entrenamiento por early stopping (sin mejora en {patience} épocas).")
            break

# Entrenamiento final con todos los datos
print("\nEntrenamiento final con todo el conjunto de entrenamiento + validación...")

from torch.utils.data import ConcatDataset
full_loader = torch.utils.data.DataLoader(
    ConcatDataset([X_train_resnet, X_val_resnet]),
    batch_size=batch_size, shuffle=True
)

# Cargar el mejor modelo antes de fine-tuning
detector.autoencoder.load_state_dict(torch.load("autoencoder_resnet_best.pth"))

final_epochs = 3
for epoch in range(final_epochs):
    loss = detector._train_epoch(full_loader, lr=learning_rate)
    print(f"Fine-tuning época {epoch+1}/{final_epochs}, pérdida: {loss:.6f}")

# Guardar modelo final
final_model_path_resnet = "autoencoder_resnet_final.pth"
torch.save(detector.autoencoder.state_dict(), final_model_path_resnet)
print(f"\nModelo final guardado en {final_model_path_resnet}")

detector.compute_threshold(train_loader)

# Evaluar ejemplos de test
print("\nEvaluando ejemplos de test...")
preds, errors = detector.eval(test_loader)
resultados=[]
# Mostrar resultados
for i, (pred, err) in enumerate(zip(preds, errors)):
    tipo = "Anomalía" if pred == 1 else "Normal"
    print(f"Ejemplo {i}: Error = {err:.4f}, Predicción = {tipo}")


# Calcular métricas
acc = accuracy_score(y_test, preds)
f1 = f1_score(y_test, preds)
roc= roc_auc_score(y_test,errors)
print(f"\nAccuracy: {acc:.4f}, F1 Score: {f1:.4f}, Area bajo la curva ROC {roc:.4f}")

resultados.append({
                'accuracy': acc,
                'f1_score': f1,
                'roc_auc': roc
            })
# Crear lista de pares (distancia, etiqueta)
error_label_pairs = list(zip(errors, y_test))

# Ordenar por distancia descendente
error_label_pairs_sorted = sorted(error_label_pairs, key=lambda x: x[0], reverse=True)

print("Distancias ordenadas (de mayor a menor) con sus etiquetas:")
for error, label in error_label_pairs_sorted:
    print(f"Distancia: {error:.4f}, Etiqueta: {label}")

folder_name=f"experimento_real/{objeto}/reconstrucción"

os.makedirs(folder_name, exist_ok=True)

# Guardar resultados en Excel
df_resultados = pd.DataFrame(resultados)
df_resultados.to_excel(os.path.join(folder_name, 'metricas_por_k.xlsx'), index=False)

# Guardar etiquetas verdaderas
pd.DataFrame({'y_test': y_test}).to_csv(os.path.join(folder_name, 'y_test.csv'), index=False)

print(f"Resultados guardados en la carpeta: {folder_name}")



/opt/python/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/python/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Entrenando Autoencoder con validación y early stopping...


/opt/python/3.10/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Época [1/100] - Pérdida entrenamiento: 0.102840 | Pérdida validación: 0.097914
Época [2/100] - Pérdida entrenamiento: 0.101692 | Pérdida validación: 0.096913
Época [3/100] - Pérdida entrenamiento: 0.099279 | Pérdida validación: 0.091104
Época [4/100] - Pérdida entrenamiento: 0.092562 | Pérdida validación: 0.090174
Época [5/100] - Pérdida entrenamiento: 0.091271 | Pérdida validación: 0.087256
Época [6/100] - Pérdida entrenamiento: 0.088109 | Pérdida validación: 0.085682
Época [7/100] - Pérdida entrenamiento: 0.089506 | Pérdida validación: 0.085096
Época [8/100] - Pérdida entrenamiento: 0.081271 | Pérdida validación: 0.085453
Época [9/100] - Pérdida entrenamiento: 0.085179 | Pérdida validación: 0.083878
Época [10/100] - Pérdida entrenamiento: 0.080258 | Pérdida validación: 0.085764
Época [11/100] - Pérdida entrenamiento: 0.083036 | Pérdida validación: 0.083183
Época [12/100] - Pérdida entrenamiento: 0.080015 | Pérdida validación: 0.085139
Época [13/100] - Pérdida entrenamiento: 0.083397 

/var/tmp/ipykernel_15146/3061866791.py:67: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  detector.autoencoder.load_state_dict(torch.load("autoencoder_resnet_best.pth"))


Fine-tuning época 1/3, pérdida: 0.063454
Fine-tuning época 2/3, pérdida: 0.062374
Fine-tuning época 3/3, pérdida: 0.062035

Modelo final guardado en autoencoder_resnet_final.pth
Umbral de reconstrucción (media + 1.96*std): 0.097502

Evaluando ejemplos de test...
Ejemplo 0: Error = 0.0618, Predicción = Normal
Ejemplo 1: Error = 0.0656, Predicción = Normal
Ejemplo 2: Error = 0.0518, Predicción = Normal
Ejemplo 3: Error = 0.0612, Predicción = Normal
Ejemplo 4: Error = 0.0797, Predicción = Normal
Ejemplo 5: Error = 0.0619, Predicción = Normal
Ejemplo 6: Error = 0.0540, Predicción = Normal
Ejemplo 7: Error = 0.0840, Predicción = Normal
Ejemplo 8: Error = 0.0740, Predicción = Normal
Ejemplo 9: Error = 0.0699, Predicción = Normal
Ejemplo 10: Error = 0.0864, Predicción = Normal
Ejemplo 11: Error = 0.0675, Predicción = Normal
Ejemplo 12: Error = 0.0677, Predicción = Normal
Ejemplo 13: Error = 0.0552, Predicción = Normal
Ejemplo 14: Error = 0.0587, Predicción = Normal
Ejemplo 15: Error = 0.0581,

## SEMILLA ALEATORIA

In [89]:
import random

import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score
import os

# Cargar el dataset desde el archivo CSV
dataset_path = f"imagenes_reales/dataset_labels.csv"
df = pd.read_csv(dataset_path)

# Mostrar las primeras filas del dataset
print(df.head(52))
random.seed(42)  # Para reproducibilidad

# Dividir el dataset en conjuntos de entrenamiento y prueba
X = df["filename"]
y = df["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

preds=[]

for i in range(len(y_test)):
    y_ejemplo = [random.randint(0,1) for _ in range(1)]  # 5 números entre 0 y 10
    preds+=y_ejemplo 

print(preds[0:50])
resultados=[]

# Calcular métricas
acc = accuracy_score(y_test, preds)
f1 = f1_score(y_test, preds)
roc= roc_auc_score(y_test,preds)
print(f"\nAccuracy: {acc:.4f}, F1 Score: {f1:.4f}, Area bajo la curva ROC {roc:.4f}")

resultados.append({
                'accuracy': acc,
                'f1_score': f1,
                'roc_auc': roc
            })

folder_name=f"experimento_real/{objeto}/semilla_aleatoria"

os.makedirs(folder_name, exist_ok=True)

# Guardar resultados en Excel
df_resultados = pd.DataFrame(resultados)
df_resultados.to_excel(os.path.join(folder_name, 'metricas_por_k.xlsx'), index=False)





               filename  label
0   train_img_00000.png      0
1   train_img_00001.png      0
2   train_img_00002.png      0
3   train_img_00003.png      0
4   train_img_00004.png      0
5   train_img_00005.png      0
6   train_img_00006.png      0
7   train_img_00007.png      0
8   train_img_00008.png      0
9   train_img_00009.png      0
10  train_img_00010.png      0
11  train_img_00011.png      0
12  train_img_00012.png      0
13  train_img_00013.png      0
14  train_img_00014.png      0
15  train_img_00015.png      0
16  train_img_00016.png      0
17  train_img_00017.png      0
18  train_img_00018.png      0
19  train_img_00019.png      0
20  train_img_00020.png      0
21  train_img_00021.png      0
22  train_img_00022.png      0
23  train_img_00023.png      0
24  train_img_00024.png      0
25  train_img_00025.png      0
26  train_img_00026.png      0
27  train_img_00027.png      0
28  train_img_00028.png      0
29  train_img_00029.png      0
30  train_img_00030.png      0
31  trai

## ZeroR

In [90]:
import random

import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score
import numpy as np

# Cargar el dataset desde el archivo CSV
dataset_path = f"imagenes_reales/dataset_labels.csv"
df = pd.read_csv(dataset_path)

# Mostrar las primeras filas del dataset
print(df.head(52))
random.seed(42)  # Para reproducibilidad

# Dividir el dataset en conjuntos de entrenamiento y prueba
X = df["filename"]
y = df["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


preds= np.zeros(len(y_test))
resultados=[]

# Calcular métricas
acc = accuracy_score(y_test, preds)
f1 = f1_score(y_test, preds)
roc= roc_auc_score(y_test,preds)
print(f"\nAccuracy: {acc:.4f}, F1 Score: {f1:.4f}, Area bajo la curva ROC {roc:.4f}")

resultados.append({
                'accuracy': acc,
                'f1_score': f1,
                'roc_auc': roc
            })

folder_name=f"experimento_real/{objeto}/ZeroR"

os.makedirs(folder_name, exist_ok=True)

# Guardar resultados en Excel
df_resultados = pd.DataFrame(resultados)
df_resultados.to_excel(os.path.join(folder_name, 'metricas_por_k.xlsx'), index=False)


               filename  label
0   train_img_00000.png      0
1   train_img_00001.png      0
2   train_img_00002.png      0
3   train_img_00003.png      0
4   train_img_00004.png      0
5   train_img_00005.png      0
6   train_img_00006.png      0
7   train_img_00007.png      0
8   train_img_00008.png      0
9   train_img_00009.png      0
10  train_img_00010.png      0
11  train_img_00011.png      0
12  train_img_00012.png      0
13  train_img_00013.png      0
14  train_img_00014.png      0
15  train_img_00015.png      0
16  train_img_00016.png      0
17  train_img_00017.png      0
18  train_img_00018.png      0
19  train_img_00019.png      0
20  train_img_00020.png      0
21  train_img_00021.png      0
22  train_img_00022.png      0
23  train_img_00023.png      0
24  train_img_00024.png      0
25  train_img_00025.png      0
26  train_img_00026.png      0
27  train_img_00027.png      0
28  train_img_00028.png      0
29  train_img_00029.png      0
30  train_img_00030.png      0
31  trai